#### Goal is to create reasonable chatbots that can choose and order the steps necessary for completing a task given a set of tools

#### TERMINOLOGIES

### Tools - Langchain interfaces, json input and corresponding functions as per their name (intuitive name and description shiuld be there)
Eg - Wikipedia- A wrapper around wikipedia, useful for answerin general knowledge questions. Input should be a search query. {describe a input schema}
### Tool kits - Collection of tools to be used togeather (github toolkit - fetching issues, posting comments, create a new file in repo)
### Agents - given a task and tools, they chose how to get the work done, these are specially constructed chains (prompt template, llm, agent output parser(return agent action obj, list of agent actions, agent finish object), not responsible for executing the actions)
### Agent Executors (Invoke the agent by user query, ouput as described above from agent,result in next action, executor checks and does as per agent action object(an obsrvation), next action variable obtained, if agent  finish return to user)

# goal is to pipe tools in a chain, create a tool from retiever, then agents and agent executors

In [3]:
# Wikipedia tool
%pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11705 sha256=8e82fe91ae35a99c95463b6fa0dffe8f20b7cdfdb458eeb743acbfffd9ad08f4
  Stored in directory: c:\users\msi\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext dotenv
%dotenv
import os
cohere_api_key = os.getenv('COHERE_API_KEY')

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_cohere import ChatCohere

from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [6]:
wikipedia_api = WikipediaAPIWrapper()
# fetch articles through load() and page summaries through run()
print(wikipedia_api.run('Python'))

f:\Users\MSI\miniconda3\envs\lanchain_env\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file f:\Users\MSI\miniconda3\envs\lanchain_env\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.
Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.



Page: Mon

In [ ]:
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_api)

In [9]:
wikipedia_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [10]:
response = wikipedia_tool.invoke({'query': 'python'})

f:\Users\MSI\miniconda3\envs\lanchain_env\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file f:\Users\MSI\miniconda3\envs\lanchain_env\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [11]:
response

'Page: Python (programming language)\nSummary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.\nPython is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.\nPython consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.\n\n\n\n

In [12]:
# pipe a tool to a simple chain
TEMPLATE = '''
Turn the following user input into a wikipedia search query. Don't answer the question:
{input}
'''
prompt_template = PromptTemplate.from_template(template = TEMPLATE)

In [13]:
from langchain_cohere import ChatCohere
chat = ChatCohere(cohere_api_key=cohere_api_key)

In [14]:
chain = prompt_template | chat | StrOutputParser()

In [15]:
chain.invoke({'input':'Who is the creator of the python programming language?'})

'Python programming language creator'

In [16]:
chain = prompt_template | chat | StrOutputParser() | wikipedia_tool

In [17]:
chain.invoke({'input':'Who is the creator of the python programming language?'})

'Page: Python (programming language)\nSummary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.\nPython is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.\nPython consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.\n\n\n\n

In [18]:
from platform import python_version
from langchain_core.tools import tool
from langchain_core.tools import create_retriever_tool

In [20]:
from langchain_community.vectorstores import Chroma

In [19]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-english-v3.0",
)

In [24]:
vectorstore = Chroma(persist_directory="./intro_to_ds",
                     embedding_function=embeddings)

In [25]:
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k':3,'lambda_mult':0.7})

In [53]:
retriever_tool = create_retriever_tool(retriever=retriever,name='introduction_to_data_and_data_science_course_lectures',description="For any questions related to data and data science course, you must use this tool")

In [54]:
retriever_tool

Tool(name='introduction_to_data_and_data_science_course_lectures', description='For any questions related to data and data science course, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001A8DE0551B0>, retriever=VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A8FA0BD420>, search_type='mmr', search_kwargs={'k': 3, 'lambda_mult': 0.7}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001A8DE0557E0>, retriever=VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A8FA0BD420>, search_type='mmr', search_kwargs={'k': 3, 'lambda_mult': 0

In [55]:
retriever_tool.args

{'query': {'description': 'query to look up in retriever',
  'title': 'Query',
  'type': 'string'}}

In [56]:
retriever_tool.invoke("list prog languages that a data scientist should know")

'Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs\n\nWhat about big data? Apart from R and Python, people working in this area are often proficient in other languages like Java or Scala. These two have not been developed specifically for doing statistical analyses, however they turn out to be very useful when combining data from multiple sources. All right! Let’s finish off with machine learning. When it comes to machine learning, we often deal with big data\n\nGreat! We hope we gave you a good idea about the level of applicability of the most frequently used program

In [32]:
@tool
def get_python_func() -> str:
    '''Useful for returning the versions of python currently used'''
    return python_version()

In [34]:
get_python_func

StructuredTool(name='get_python_func', description='Useful for returning the versions of python currently used', args_schema=<class 'langchain_core.utils.pydantic.get_python_func'>, func=<function get_python_func at 0x000001A8F99AAC20>)

In [35]:
get_python_func.args

{}

In [36]:
get_python_func.invoke({})

'3.10.10'

In [ ]:
# Now to create an agent we need - Prompt Template, LLM, List of tools
# tools in hand - wiki, custom, retriever



In [37]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain import hub

In [57]:
tools = [wikipedia_tool, retriever_tool, get_python_func]

In [40]:
chat_prompt_template = hub.pull('hwchase17/openai-tools-agent')

f:\Users\MSI\miniconda3\envs\lanchain_env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
chat_prompt_template.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [44]:
# all langchain agent prompt templates available in langchain website

In [46]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [58]:
agent = create_tool_calling_agent(llm=chat,tools=tools,prompt=chat_prompt_template)
agent_executor = AgentExecutor(agent = agent, tools= tools,
                               verbose=True,
                               return_intermediate_steps=True)

In [59]:
agent_executor.invoke({'input':'tell me the version of python Im using'})



> Entering new AgentExecutor chain...

Invoking: `get_python_func` with `{}`
responded: I will use the get_python_func tool to find out the version of Python the user is using.

3.10.10You are using Python version 3.10.10.

> Finished chain.


{'input': 'tell me the version of python Im using',
 'output': 'You are using Python version 3.10.10.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_func', tool_input={}, log='\nInvoking: `get_python_func` with `{}`\nresponded: I will use the get_python_func tool to find out the version of Python the user is using.\n\n', message_log=[AIMessageChunk(content='I will use the get_python_func tool to find out the version of Python the user is using.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '69b849db-9a39-447d-8ca2-2352b555bced', 'tool_calls': [{'id': 'bf6f1fb334454ceca999cb4e205b3756', 'function': {'name': 'get_python_func', 'arguments': '{}'}, 'type': 'function'}], 'token_count': {'input_tokens': 1045.0, 'output_tokens': 57.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generat

In [60]:
response = agent_executor.invoke({'input':'Could you list the programming languages listed in the introduction to data and data science course and also list the names of their creators.End the response with a hilarious topic related joke'})



> Entering new AgentExecutor chain...

Invoking: `introduction_to_data_and_data_science_course_lectures` with `{'query': 'programming languages'}`
responded: I will search for the programming languages listed in the introduction to data and data science course and also list the names of their creators. Then I will search for a hilarious topic related joke.

Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs

More importantly, it will be sufficient for your need to create quick and accurate analyses. However, if your theoretical preparation is strong enough, you will 

In [64]:
# The output of agents:ToolAgentAction (each intermediate step consists of action and corresponding result)
response['intermediate_steps']

[(ToolAgentAction(tool='introduction_to_data_and_data_science_course_lectures', tool_input={'query': 'programming languages'}, log="\nInvoking: `introduction_to_data_and_data_science_course_lectures` with `{'query': 'programming languages'}`\nresponded: I will search for the programming languages listed in the introduction to data and data science course and also list the names of their creators. Then I will search for a hilarious topic related joke.\n\n", message_log=[AIMessageChunk(content='I will search for the programming languages listed in the introduction to data and data science course and also list the names of their creators. Then I will search for a hilarious topic related joke.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '70677684-8183-4199-8102-9702f2744221', 'tool_calls': [{'id': 'ed1abaeca33f4e2ca09e1f0a8a180eab', 'function': {'name': 'introduction_to_data_and_data

In [68]:
response = agent_executor.invoke({'input':'tell me the python version'})



> Entering new AgentExecutor chain...

Invoking: `get_python_func` with `{}`
responded: I will use the get_python_func tool to find the Python version.

3.10.10The Python version is 3.10.10.

> Finished chain.


In [67]:
response

[ToolAgentAction(tool='get_python_func', tool_input={}, log="\nInvoking: `get_python_func` with `{}`\nresponded: I will use the 'get_python_func' tool to find out the Python version.\n\n", message_log=[AIMessage(content="I will use the 'get_python_func' tool to find out the Python version.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'd3debe1f-1188-4107-8ccd-2672904993d1', 'tool_calls': [{'id': '778a3ec52cdc47b5981307e4dae8a50b', 'function': {'name': 'get_python_func', 'arguments': '{}'}, 'type': 'function'}], 'token_count': {'input_tokens': 1042.0, 'output_tokens': 54.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'd3debe1f-1188-4107-8ccd-2672904993d1', 'tool_calls': [{'id': '778a3ec52cdc47b5981307e4dae8a50b', 'function': {'name': 'get_python_func', 'arguments': '{}'}, 'typ

In [69]:
agent.invoke({'input':'tell me the python version', 'intermediate_steps':response['intermediate_steps']})

AgentFinish(return_values={'output': 'The Python version is 3.10.10.'}, log='The Python version is 3.10.10.')

In [ ]:
# Agent Finish means no further steps needs to be taken